In [1]:
import time
from termcolor import colored
import torch
import torch.autograd.profiler as profiler

from modules.Dcls2d import Dcls2d
from modules.Dcls2dFull import Dcls2dFull


assert torch.cuda.is_available()
cuda_device = torch.device("cuda")  # device object representing GPU

in_channels = 1
out_channels = 1
kernel_size = (2,2)
dilation = (2,2)
stride = (1,1)
padding = (0,0)
groups = 1
bias = False

m = torch.nn.Conv2d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)

n = Dcls2d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)

o = Dcls2dFull(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)

X1 = torch.nn.Parameter(
                      torch.tensor([[[[1., 2., 3., 4.],
                                    [5., 6., 7., 8.], 
                                    [9., 10., 11., 12.],
                                    [13., 14., 15., 16.]]]],device=cuda_device),
                      requires_grad = True) 
X2 = torch.nn.Parameter(
                      torch.tensor([[[[1., 2., 3., 4.],
                                    [5., 6., 7., 8.], 
                                    [9., 10., 11., 12.],
                                    [13., 14., 15., 16.]]]],device=cuda_device),
                      requires_grad = True) 
X3 = torch.nn.Parameter(
                      torch.tensor([[[[1., 2., 3., 4.],
                                    [5., 6., 7., 8.], 
                                    [9., 10., 11., 12.],
                                    [13., 14., 15., 16.]]]],device=cuda_device),
                      requires_grad = True) 
m.weight = torch.nn.Parameter(
                      torch.tensor([[[[20., 40.],
                                    [60., 80.]]]],device=cuda_device),
                      requires_grad = True)
n.weight = torch.nn.Parameter(
                      torch.tensor([[[[20., 40.],
                                    [60., 80.]]]],device=cuda_device),
                      requires_grad = True)
o.weight = torch.nn.Parameter(
                      torch.tensor([[[[20., 40.],
                                    [60., 80.]]]],device=cuda_device),
                      requires_grad = True)

height_out = (4 + 2 * padding[0] - (dilation[0] * (kernel_size[0] - 1) + 1)) / stride[0] + 1;
print((dilation[0] * (kernel_size[0] - 1) + 1))
width_out = (4 + 2 * padding[1] - (dilation[1] * (kernel_size[1] - 1) + 1)) / stride[1] + 1;
back_truth = torch.nn.Parameter(
                      torch.tensor([[[[1., 2.],
                                    [4., 5.]]]],device=cuda_device),
                      requires_grad = True)

with torch.autograd.profiler.profile(use_cuda=True, profile_memory=True) as prof:
    var2 = (n(X2) - back_truth).norm()
var3 = (o(X3) - back_truth).norm()
var1 = (m(X1) - back_truth).norm()

var1.backward();
var2.backward();
var3.backward();

3


In [2]:
print(X1.size())
print(m.weight.size())
print(n.weight.size())
print(o.weight.size())

print(m(X1).size())
print(m(X1))
print(n(X2).size())
print(n(X2))
print(o(X2).size())
print(o(X2))


print(m.weight.grad) 
print(n.weight.grad)
print(o.weight.grad)

torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 2, 2])
tensor([[[[1560., 1760.],
          [2360., 2560.]]]], device='cuda:0',
       grad_fn=<CudnnConvolutionBackward>)
torch.Size([1, 1, 2, 2])
tensor([[[[1559.9611, 1759.9611],
          [2359.9612, 2559.9612]]]], device='cuda:0',
       grad_fn=<SurrogateDilationBackward>)
torch.Size([1, 1, 2, 2])
tensor([[[[1559.8484, 1759.8484],
          [2359.8484, 2559.8484]]]], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[[[ 7.6718, 11.5944],
          [23.3621, 27.2847]]]], device='cuda:0')
tensor([[[[ 7.6719, 11.5944],
          [23.3621, 27.2847]]]], device='cuda:0')
tensor([[[[ 7.6719, 11.5944],
          [23.3621, 27.2847]]]], device='cuda:0')


In [3]:
R1 = torch.ceil(n.P1) - n.P1
R2 = torch.ceil(n.P2) - n.P2

print((1 - R1) * (1-R2))
print((R1) * (1-R2))
print((1 - R1) * (R2))
print((R1) * (R2))

tensor([[[1., 1.],
         [1., 1.]]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([[[0., 0.],
         [0., 0.]]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([[[0., 0.],
         [0., 0.]]], device='cuda:0', grad_fn=<MulBackward0>)
tensor([[[0., 0.],
         [0., 0.]]], device='cuda:0', grad_fn=<MulBackward0>)


In [4]:
print(o.P1)

Parameter containing:
tensor([[[[-2., -2.],
          [ 0.,  0.]]]], device='cuda:0', requires_grad=True)


In [5]:
torch.cuda.memory_summary(device=cuda_device, abbreviated=True)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   15872 B  |   27136 B  |  366080 B  |  350208 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |   15872 B  |   27136 B  |  366080 B  |  350208 B  |\n|---------------------------------------------------------------------------|\n| GPU reserved memory   |    2048 KB |    2048 KB |    2048 KB |       0 B  |\n|--------------------------------------------------

In [6]:
print(prof.key_averages().table(sort_by="self_cuda_mem", row_limit=100))

AttributeError: 'FunctionEventAvg' object has no attribute 'self_cuda_mem'

In [ ]:
------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                    Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
       SurrogateDilation         7.18%     348.164us        96.64%       4.685ms       4.685ms       2.399ms        48.21%       4.945ms       4.945ms           0 b           0 b         512 b     -10.00 Kb             1  
             aten::addmm        73.38%       3.557ms        73.56%       3.566ms       3.566ms       2.335ms        46.92%       2.335ms       2.335ms           0 b           0 b         512 b           0 b             1  
             aten::clamp         2.22%     107.562us         4.04%     195.795us      24.474us      42.400us         0.85%      60.704us       7.588us           0 b           0 b       4.00 Kb           0 b             8  
               aten::sub         2.74%     132.760us         3.38%     163.643us      23.378us      52.320us         1.05%      52.320us       7.474us           0 b           0 b       3.50 Kb           0 b             7  
             aten::floor         2.52%     121.967us         5.08%     246.098us      61.524us      24.160us         0.49%      35.232us       8.808us           0 b           0 b       2.00 Kb           0 b             4  
            aten::select         1.63%      78.892us         1.76%      85.474us      12.211us      28.672us         0.58%      28.672us       4.096us           0 b           0 b           0 b           0 b             7  
               aten::mul         1.46%      70.946us         1.79%      86.690us      21.673us      26.784us         0.54%      26.784us       6.696us           0 b           0 b       2.00 Kb           0 b             4  
    aten::frobenius_norm         0.54%      26.065us         2.62%     126.994us     126.994us       5.856us         0.12%      24.928us      24.928us           0 b           0 b       1.00 Kb           0 b             1  
             aten::copy_         1.61%      78.109us         1.61%      78.109us      26.036us      21.567us         0.43%      21.567us       7.189us           0 b           0 b           0 b           0 b             3  
              aten::add_         1.16%      56.354us         1.16%      56.354us      28.177us      19.200us         0.39%      19.200us       9.600us           0 b           0 b           0 b           0 b             2  
              aten::norm         1.22%      59.353us         1.39%      67.418us      67.418us      10.880us         0.22%      10.880us      10.880us           0 b           0 b         512 b           0 b             1  
         aten::ones_like         0.25%      11.891us         1.00%      48.387us      48.387us       4.256us         0.09%      10.240us      10.240us           0 b           0 b         512 b           0 b             1  
             aten::fill_         0.51%      24.793us         0.51%      24.793us      24.793us       5.984us         0.12%       5.984us       5.984us           0 b           0 b           0 b           0 b             1  
             aten::empty         1.71%      82.755us         1.71%      82.755us       3.598us       0.000us         0.00%       0.000us       0.000us           0 b           0 b       7.50 Kb       7.50 Kb            23  
           aten::resize_         0.96%      46.379us         0.96%      46.379us       5.797us       0.000us         0.00%       0.000us       0.000us           0 b           0 b       4.00 Kb       4.00 Kb             8  
        aten::empty_like         0.09%       4.601us         0.24%      11.703us      11.703us       0.000us         0.00%       0.000us       0.000us           0 b           0 b         512 b           0 b             1  
     aten::empty_strided         0.15%       7.102us         0.15%       7.102us       7.102us       0.000us         0.00%       0.000us       0.000us           0 b           0 b         512 b         512 b             1  
              aten::view         0.44%      21.254us         0.44%      21.254us       3.036us       0.000us         0.00%       0.000us       0.000us           0 b           0 b           0 b           0 b             7  
        aten::as_strided         0.18%       8.892us         0.18%       8.892us       0.988us       0.000us         0.00%       0.000us       0.000us           0 b           0 b           0 b           0 b             9  
            aten::expand         0.03%       1.558us         0.05%       2.351us       2.351us       0.000us         0.00%       0.000us       0.000us           0 b           0 b           0 b           0 b             1  
            aten::stride         0.02%       1.079us         0.02%       1.079us       0.360us       0.000us         0.00%       0.000us       0.000us           0 b           0 b           0 b           0 b             3  
------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 4.848ms
CUDA time total: 4.976ms

